SPRAWDZIC WYJSCIE Z NASZEJ FPN JESLI DAMY INNY TENSOR NA WEJSCIE 640x640
I RESNETA tez czy zmienia sie liczba paraemtrow

In [4]:
import torch
from torchsummary import summary
from model.ResNet50 import ResNet50

In [14]:
PATH_MODEL = "data/model/"

# ResNet50

In [5]:
model = ResNet50()

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\jkwia/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth
100%|█████████████████████████████████████████████████████████████████████████████| 97.8M/97.8M [00:43<00:00, 2.37MB/s]


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

C:\Users\jkwia\anaconda3\lib\site-packages\torch\cuda\__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at C:\cb\pytorch_1000000000000\work\c10\cuda\CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [8]:
device

device(type='cpu')

In [7]:
model.to(device)

ResNet50(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1

In [10]:
input_tensor = torch.randn(1,3, 512, 800).to(device)
output = model(input_tensor)
print(output.shape)

torch.Size([1, 2048, 16, 25])


In [7]:
summary(model, (3, 224, 224),device=device.type)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [11]:
summary(model, (3, 512, 800),device=device.type)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 400]           9,408
       BatchNorm2d-2         [-1, 64, 256, 400]             128
              ReLU-3         [-1, 64, 256, 400]               0
         MaxPool2d-4         [-1, 64, 128, 200]               0
            Conv2d-5         [-1, 64, 128, 200]           4,096
       BatchNorm2d-6         [-1, 64, 128, 200]             128
              ReLU-7         [-1, 64, 128, 200]               0
            Conv2d-8         [-1, 64, 128, 200]          36,864
       BatchNorm2d-9         [-1, 64, 128, 200]             128
             ReLU-10         [-1, 64, 128, 200]               0
           Conv2d-11        [-1, 256, 128, 200]          16,384
      BatchNorm2d-12        [-1, 256, 128, 200]             512
           Conv2d-13        [-1, 256, 128, 200]          16,384
      BatchNorm2d-14        [-1, 256, 1

In [8]:
torch.save(model.state_dict(), PATH_MODEL+'model.pth')

# FPN

In [13]:
from model.FeaturesPyramid import FeaturesPyramid

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
in_channels_list = [512, 1024, 2048]
out_channels = 256
img_size_c3 = 56
inputs = []
inputs = [torch.randn(1, c, img_size_c3//(2**i), img_size_c3//(2**i)).to(device) for i, c in enumerate(in_channels_list, start=1)]

# Utwórz instancję klasy FeaturesPyramid
fpn = FeaturesPyramid(in_channels_list, out_channels).to(device)

# Wywołaj funkcję forward na danych wejściowych
outputs = fpn(inputs)

# Wyświetl wyniki
for i, output in enumerate(outputs):
    print(f'Output P{i + 3} shape: {output.shape}')

Output P3 shape: torch.Size([1, 256, 28, 28])
Output P4 shape: torch.Size([1, 256, 14, 14])
Output P5 shape: torch.Size([1, 256, 7, 7])
Output P6 shape: torch.Size([1, 256, 4, 4])
Output P7 shape: torch.Size([1, 256, 2, 2])


In [5]:
inputs[0].size(),inputs[1].size(),inputs[2].size()

(torch.Size([1, 512, 28, 28]),
 torch.Size([1, 1024, 14, 14]),
 torch.Size([1, 2048, 7, 7]))

In [6]:
s0 = tuple(inputs[0].size()); s1=tuple(inputs[1].size()); s2=tuple(inputs[2].size())
s0,s1,s2, type(s0), type(s0[0])

((1, 512, 28, 28), (1, 1024, 14, 14), (1, 2048, 7, 7), tuple, int)

In [19]:
# https://github.com/sksq96/pytorch-summary
# summary(fpn,input_size=[s0,s1,s2],device=device.type)
# summary(fpn,input_size=inputs,device=device.type)

# RetinaNet

In [31]:
from model.RetinaNet import RetinaNet

In [36]:
RetNet = RetinaNet().to(device)

In [37]:
input_tensor = torch.randn(1,3, 512, 800).to(device)

In [38]:
input_tensor.size()

torch.Size([1, 3, 512, 800])

In [39]:
outputs_RN = RetNet(input_tensor)

In [40]:
for i, output in enumerate(outputs_RN):
    print(f'Output P{i + 3} shape: {output.shape}')

TypeError: 'NoneType' object is not iterable